In [9]:
import csv
import numpy as np

slide_matching_path = "/mnt/scratchc/fmlab/zuberi01/masters/slide_matching.csv"
model_training_input_path = "/mnt/scratchc/fmlab/zuberi01/masters/model_training_input_230815.csv"

slide_matching = []
model_training_input = []

with open(slide_matching_path, mode='r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        slide_matching.append(lines)

with open(model_training_input_path, mode='r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        model_training_input.append(lines)

slide_matching_PatientID_index = slide_matching[0].index('PatientID')
slide_matching_all_PatientIDs = []
for i in range(1, len(slide_matching)):
    id = slide_matching[i][slide_matching_PatientID_index]
    if id != '':
        slide_matching_all_PatientIDs.append(int(float(id)))
#print(np.unique(slide_matching_all_PatientIDs, return_counts=True)[1])

model_training_input_PatientID_index = model_training_input[0].index('Patient ID')
model_training_input_all_PatientIDs = []
for i in range(1, len(model_training_input)):
    id = model_training_input[i][model_training_input_PatientID_index]
    if id != '':
        model_training_input_all_PatientIDs.append(int(float(id)))

#print(np.unique(model_training_input_all_PatientIDs, return_counts=True)[1])

# check in slide_matching there are any duplicate rows
# if num of rows and num of unique is the same then there are no duplicates
#print(len(slide_matching))
#print(len(set([tuple(i) for i in slide_matching])))
#print(len(set([tuple(i) for i in model_training_input])))
#print(len(model_training_input))

slide_matching_year_index = slide_matching[0].index('EndoscopyDate')
model_training_input_months_before_final_index = model_training_input[0].index('Months before final')

slide_matching_block_index = slide_matching[0].index('Block')
model_training_input_esophageal_location_index = model_training_input[0].index('Esophageal location')

def match_endoscopies_with_indices(list1, list2):
        # Sort list1 by year (descending) and list2 by months (ascending)
        #print(len(list1),len(list2))
        list1_sorted = enumerate(list1, start=1)
        list2_sorted = enumerate(list2, start=1)
        
        # Initialize the list for matches
        matches = []
        used_indices = set()
        
        # Match entries from list1 and list2 starting from the largest year to the smallest with the smallest month to the largest
        for i, (year, location1) in list1_sorted:
                if location1 == '':
                        continue
                for j, (months, location2) in list2_sorted:
                        if j not in used_indices and location1 == location2:
                                matches.append((i, j))
                                used_indices.add(j)
                                break
        
        # Add edge cases for unmatched entries
        unmatched_list1 = [i for i, (year, location1) in list1_sorted if location1 == '' or not any(location1 == loc for _, (months, loc) in list2_sorted)]
        unmatched_list2 = [j for j, (months, location2) in list2_sorted if j not in used_indices]

        # now if there are duplicates in the list1_sorted or list2_sorted we remove those matches
        elements_of_list1 = [list1[matches[i][0]-1] for i in range(len(matches))]
        elements_of_list2 = [list2[matches[i][1]-1] for i in range(len(matches))]

        #check if there are duplicates in list1 and get their indices
        duplicates_list1 = []

        matches_elements_and_indices = []
        for match in matches:
                #print(match)
                list1_match = list1[match[0]-1]
                first = [list1_match, match[0]-1]

                list2_match = list2[match[1]-1]
                second = [list2_match, match[1]-1]

                matches_elements_and_indices.append((first,second))
                
        #print(matches_elements_and_indices)
        
        #print(elements_of_list1)
        #print(elements_of_list2)
        return matches

# create a new CSV file which will have the matching rows from slide_matching and model_training_input
new_csv = []
# add as the first row the headers from slide_matching and model_training_input
slide_matching_renamed_rows = []
for i in slide_matching[0]:
    slide_matching_renamed_rows.append(i + '_SM')

model_training_input_renamed_rows = []
for i in model_training_input[0]:
    model_training_input_renamed_rows.append(i + '_MT230815')

new_csv.append(slide_matching_renamed_rows + model_training_input_renamed_rows)

print(len(new_csv))

# take each row in model_training_input and find its match
# we have to separate them per Patient ID
rows_excluded = 0
rows_in = 0
index_for_all = 1
num_matches = 0
current_row_in_new_csv = 1

pat_ids = 0
for patient_id in np.unique(model_training_input_all_PatientIDs):
    # get the rows in slide_matching that have the same patient_id
    slide_matching_rows = []
    for i in range(1, len(slide_matching)):
        if slide_matching[i][slide_matching_PatientID_index] != '' and int(float(slide_matching[i][slide_matching_PatientID_index])) == patient_id:
            row = slide_matching[i]

            # now we will change the date format to just be the year and be an int
            row[slide_matching_year_index] = int(float(row[slide_matching_year_index][:4]))

            slide_matching_rows.append(row)
    # sort slide_matching_rows by the years
    slide_matching_rows = sorted(slide_matching_rows, key=lambda x: x[slide_matching_year_index], reverse=False)
    # print just the patient_id and the years
    slide_matching_rows_years = []
    for i in slide_matching_rows:
        slide_matching_rows_years.append(i[slide_matching_year_index])

    #print(slide_matching_rows)

    # get the row in model_training_input that has the same patient_id
    model_training_input_rows = []
    for i in range(1, len(model_training_input)):
        if model_training_input[i][model_training_input_PatientID_index] != '' and int(float(model_training_input[i][model_training_input_PatientID_index])) == patient_id:
            
            row = model_training_input[i]

            # now we will change the date format to just be the year and be an int
            row[model_training_input_months_before_final_index] = int(float(row[model_training_input_months_before_final_index]))

            model_training_input_rows.append(model_training_input[i])
    # sort model_training_input_rows by the months before final
    model_training_input_rows = sorted(model_training_input_rows, key=lambda x: x[model_training_input_months_before_final_index], reverse=True)
    model_training_input_rows_months_before_final = []
    for i in model_training_input_rows:
        model_training_input_rows_months_before_final.append(i[model_training_input_months_before_final_index])

    

    #rows_in += len(slide_matching_rows)
    #if len(slide_matching_rows) != len(set(slide_matching_rows_years)) and len(model_training_input_rows) != len(set(model_training_input_rows_months_before_final)):

        #rows_excluded += len(slide_matching_rows) 

        #print("ERROR, duplicate years in slide_matching_rows")
        #if len(model_training_input_rows) != len(set(model_training_input_rows_months_before_final)):
        #    print("ERROR, duplicate years in model_training_input_rows")

       

    #first clean up the slide_matching_rows' block locations to just keep the integers and then sort them
    #sort them by the years first and then by block locations
    for j, row in enumerate(slide_matching_rows):
        block = row[slide_matching_block_index]
        # only keep the integers and if there is nothing left print it and break
        block = ''.join(filter(str.isdigit, block))
        #if block == '':
        #    print('Block:', row[slide_matching_block_index])
        #    break
        slide_matching_rows[j][slide_matching_block_index] = block
    #slide_matching_rows = sorted(slide_matching_rows, key=lambda x: (x[slide_matching_year_index], x[slide_matching_block_index]), reverse=False)

    #sort the model_training_input_rows first by months_before_final and then by esophageal location
    #model_training_input_rows = sorted(model_training_input_rows, key=lambda x: (x[model_training_input_months_before_final_index], x[model_training_input_esophageal_location_index]), reverse=False)
    #for j in model_training_input_rows:
    #    print(j)

    #print('Patient id:', patient_id)

    slide_matching_to_match = []
    slide_matching_full_rows = []
    for j in slide_matching_rows:
        #print(j[slide_matching_block_index],type(j[slide_matching_block_index]))
        year = j[slide_matching_year_index]
        loc = j[slide_matching_block_index]
        if loc != '': loc = int(float(loc))
        slide_matching_full_rows.append(j)
        slide_matching_to_match.append((year, loc))
    #print(slide_matching_to_match)
        #print(j[slide_matching_year_index],j[slide_matching_block_index])

    model_training_to_match = []
    model_training_full_rows = []
    for j in model_training_input_rows:
        months = j[model_training_input_months_before_final_index]
        loc = j[model_training_input_esophageal_location_index]
        if loc != '': loc= int(float(loc))
        model_training_to_match.append((months,loc))
        model_training_full_rows.append(j)
        #print(j[model_training_input_months_before_final_index],j[model_training_input_esophageal_location_index])
    #print(model_training_to_match)
    #print('\n')

    #for j in range(len(slide_matching_rows)):
    #    slide_matching_rows[j].append(index_for_all)
    #    model_training_input_rows[j].append(index_for_all)
    #    index_for_all += 1

    length_of_both_avg = (len(slide_matching_to_match) + len(model_training_to_match))/2
    #print(slide_matching_to_match)
    #print(len(slide_matching_to_match),len(model_training_to_match))
    matches = match_endoscopies_with_indices(slide_matching_to_match, model_training_to_match)
    #print(len(matches))
    #for match in matches:
    #    print(slide_matching_to_match[match[0] - 1], model_training_to_match[match[1] - 1])
    num_matches += len(matches)

    # now I want to create CSV which merges every column from slide_matching.csv and model_training_input_230815.csv with every matching row put together
    # I want to rename each column by their original CSV so EndoscopySampleID becomes EndoscopySampleID_SM (SM is for slide matching)
    # Esophageal location becomes Esophageal location_MT230815 (MT230815 is for model training 230815)

    new_row_in_csv = []
    for match in matches:
        #print(slide_matching_rows[match[0] - 1])
        #print(model_training_input_rows[match[1] - 1])
        new_row_in_csv = slide_matching_full_rows[match[0] - 1] + model_training_full_rows[match[1] - 1]

        new_csv.append(new_row_in_csv)

print(pat_ids)

1
88


In [11]:
import csv
from collections import defaultdict
import numpy as np

# Define file paths
slide_matching_path = "/mnt/scratchc/fmlab/zuberi01/masters/slide_matching.csv"
model_training_input_path = "/mnt/scratchc/fmlab/zuberi01/masters/model_training_input_230815.csv"

# Initialize data lists
slide_matching = []
model_training_input = []

# Read slide_matching.csv into a list of dictionaries
with open(slide_matching_path, mode='r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        slide_matching.append(row)

# Read model_training_input_230815.csv into a list of dictionaries
with open(model_training_input_path, mode='r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        model_training_input.append(row)

# Group slide_matching rows by PatientID
slide_matching_by_patient = defaultdict(list)
for row in slide_matching:
    patient_id = row.get('PatientID', '').strip()
    if patient_id:
        try:
            patient_id_num = int(float(patient_id))
            slide_matching_by_patient[patient_id_num].append(row)
        except ValueError:
            print(f"Warning: Invalid PatientID '{patient_id}' in slide_matching")
            continue

# Group model_training_input rows by Patient ID
model_training_by_patient = defaultdict(list)
for row in model_training_input:
    patient_id = row.get('Patient ID', '').strip()
    if patient_id:
        try:
            patient_id_num = int(float(patient_id))
            model_training_by_patient[patient_id_num].append(row)
        except ValueError:
            print(f"Warning: Invalid Patient ID '{patient_id}' in model_training_input")
            continue

# Get the set of common patient IDs
common_patient_ids = set(slide_matching_by_patient.keys()).intersection(set(model_training_by_patient.keys()))

# Prepare headers for the new CSV
slide_matching_headers = slide_matching[0].keys()
slide_matching_renamed_headers = [h + '_SM' for h in slide_matching_headers]

model_training_headers = model_training_input[0].keys()
model_training_renamed_headers = [h + '_MT230815' for h in model_training_headers]

# Initialize the new CSV data
new_csv = []
new_csv.append(slide_matching_renamed_headers + model_training_renamed_headers)

def match_endoscopies_with_indices(list1, list2):
    """
    Match entries from list1 and list2 based on location.
    Each list contains tuples of (value, location).
    """
    matches = []
    used_indices = set()
    for idx1, (val1, loc1) in enumerate(list1):
        for idx2, (val2, loc2) in enumerate(list2):
            if idx2 in used_indices:
                continue
            if loc1 == loc2:
                matches.append((idx1, idx2))
                used_indices.add(idx2)
                break
    return matches

# Process each common patient ID
for patient_id in common_patient_ids:
    slide_rows = slide_matching_by_patient[patient_id]
    model_rows = model_training_by_patient[patient_id]
    
    # Prepare data for matching
    slide_data = []
    for row in slide_rows:
        # Extract and validate EndoscopyDate (year)
        endoscopy_date = row.get('EndoscopyDate', '').strip()
        try:
            year = int(endoscopy_date[:4])
        except (ValueError, IndexError):
            print(f"Warning: Invalid EndoscopyDate '{endoscopy_date}' for PatientID {patient_id}")
            continue
        # Extract and validate Block (location)
        block = row.get('Block', '').strip()
        block_digits = ''.join(filter(str.isdigit, block))
        if not block_digits:
            print(f"Warning: Invalid Block '{block}' for PatientID {patient_id}")
            continue
        location = int(block_digits)
        slide_data.append({'year': year, 'location': location, 'row': row})
    
    model_data = []
    for row in model_rows:
        # Extract and validate Months before final
        months_before_final = row.get('Months before final', '').strip()
        try:
            months = int(float(months_before_final))
        except ValueError:
            print(f"Warning: Invalid Months before final '{months_before_final}' for PatientID {patient_id}")
            continue
        # Extract and validate Esophageal location
        esophageal_location = row.get('Esophageal location', '').strip()
        loc_digits = ''.join(filter(str.isdigit, esophageal_location))
        if not loc_digits:
            print(f"Warning: Invalid Esophageal location '{esophageal_location}' for PatientID {patient_id}")
            continue
        location = int(loc_digits)
        model_data.append({'months': months, 'location': location, 'row': row})
    
    # Create lists for matching
    slide_match_list = [(item['year'], item['location']) for item in slide_data]
    model_match_list = [(item['months'], item['location']) for item in model_data]
    
    # Perform matching
    matches = match_endoscopies_with_indices(slide_match_list, model_match_list)
    
    # Combine matched rows
    for idx1, idx2 in matches:
        slide_row = slide_data[idx1]['row']
        model_row = model_data[idx2]['row']
        combined_row = [slide_row[h] for h in slide_matching_headers] + [model_row[h] for h in model_training_headers]
        new_csv.append(combined_row)

# Optionally, write the new CSV data to a file
#output_path = "/path/to/output.csv"
#with open(output_path, mode='w', newline='') as file:
#    writer = csv.writer(file)
#    writer.writerows(new_csv)

print("Data processing complete. The new CSV file has been created.")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/output.csv'

In [14]:
output_path = "matching_rows.csv"
with open(output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(new_csv)

In [15]:
import pandas as pd

slide_matching_path = "/mnt/scratchc/fmlab/zuberi01/masters/slide_matching.csv"
model_training_input_path = "/mnt/scratchc/fmlab/zuberi01/masters/model_training_input_230815.csv"


# Read the CSV files
df_model = pd.read_csv(model_training_input_path)
df_slide = pd.read_csv(slide_matching_path)

# Ensure 'Patient ID' and 'OGD_ID' in df_model are integers
df_model['Patient ID'] = pd.to_numeric(df_model['Patient ID'], errors='coerce').astype('Int64')
df_model['OGD_ID'] = pd.to_numeric(df_model['OGD_ID'], errors='coerce').astype('Int64')
df_model['Block'] = pd.to_numeric(df_model['Block'], errors='coerce').astype('Int64')

# Create 'Block_str' in df_model (e.g., 'B1', 'B2', etc.)
df_model['Block_str'] = 'B' + df_model['Block'].astype(str)

# Ensure 'PatientID' and 'EndoscopyID' in df_slide are integers
df_slide['PatientID'] = pd.to_numeric(df_slide['PatientID'], errors='coerce').astype('Int64')
df_slide['EndoscopyID'] = pd.to_numeric(df_slide['EndoscopyID'], errors='coerce').astype('Int64')

# Clean 'Block' in df_slide (ensure it's uppercase and stripped of whitespace)
df_slide['Block'] = df_slide['Block'].astype(str).str.strip().str.upper()

# Parse 'EndoscopyDate' in df_slide to datetime
df_slide['EndoscopyDate'] = pd.to_datetime(df_slide['EndoscopyDate'], errors='coerce')

# Create a dictionary to store the final dates for each patient
final_dates = {}

# Identify the final date for each patient
for patient_id in df_model['Patient ID'].unique():
    df_patient = df_model[df_model['Patient ID'] == patient_id]
    df_final = df_patient[df_patient['Months before final'] == 0]
    if not df_final.empty:
        final_ogd_ids = df_final['OGD_ID'].unique()
        for ogd_id in final_ogd_ids:
            df_slide_match = df_slide[(df_slide['PatientID'] == patient_id) & (df_slide['EndoscopyID'] == ogd_id)]
            if not df_slide_match.empty:
                endoscopy_date = df_slide_match.iloc[0]['EndoscopyDate']
                final_dates[patient_id] = endoscopy_date
                break

# Compute 'EndoscopyDate' in df_model based on 'Months before final'
def compute_endoscopy_date(row):
    patient_id = row['Patient ID']
    months_before_final = row['Months before final']
    if patient_id in final_dates and pd.notnull(final_dates[patient_id]):
        final_date = final_dates[patient_id]
        endoscopy_date = final_date - pd.DateOffset(months=months_before_final)
        return endoscopy_date
    else:
        return pd.NaT

df_model['EndoscopyDate'] = df_model.apply(compute_endoscopy_date, axis=1)

# Merge the DataFrames on 'Patient ID', 'OGD_ID', and 'Block'
df_merged = pd.merge(
    df_model,
    df_slide,
    left_on=['Patient ID', 'OGD_ID', 'Block_str'],
    right_on=['PatientID', 'EndoscopyID', 'Block'],
    how='left',
    suffixes=('_model', '_slide')
)

# Optional: If you want to match on 'EndoscopyDate' as well (allowing for exact date matches)
# Merge on 'Patient ID', 'EndoscopyDate', and 'Block'
# Note: This step assumes that dates match exactly, which might not always be the case
# df_merged = pd.merge(
#     df_model,
#     df_slide,
#     left_on=['Patient ID', 'EndoscopyDate', 'Block_str'],
#     right_on=['PatientID', 'EndoscopyDate', 'Block'],
#     how='left',
#     suffixes=('_model', '_slide')
# )

# Save the merged DataFrame to a CSV file
df_merged.to_csv('merged_output.csv', index=False)

# Display the first few rows of the merged DataFrame
print(df_merged.head())

   index  Block_model  Patient ID Status  Esophageal location Pathology2  \
0      1            1           1      P                    1       NDBE   
1      2            1           1      P                    1       NDBE   
2      3            1           1      P                    1        LGD   
3      4            1           2      P                    1       NDBE   
4      5            3           2      P                    3       NDBE   

   Months before final  OGD_ID OGD_pathology classification  ... Block_slide  \
0                  120       1          NDBE         NDBE_P  ...          B1   
1                   96       2          NDBE         NDBE_P  ...          B1   
2                   48       3           LGD          LGD_P  ...          B1   
3                   60       4          NDBE         NDBE_P  ...          B1   
4                   60       4          NDBE         NDBE_P  ...          B3   

   BlockLevel  label  p53IHC       Site  SampleType  \
0      

In [16]:
print(len(df_merged))

758
